__ProPythia__

This file is a simulation for the antioxidant dataset;

I intend to run propythia here so that in the future I can make comparisons with the results obtained with omnia.

i will do this simulation with the antioxidant data, where the data in unbalanced

# imports

In [0]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

from scipy.stats import loguniform
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, matthews_corrcoef,f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier

from propythia.protein.descriptors import ProteinDescritors
from propythia.protein.encoding import Encoding
from propythia.ml.shallow_ml import ShallowML
from propythia.protein.sequence import ReadSequence

from propythia.ml.deep_ml import DeepML

from propythia.feature_selection import FeatureSelection

from propythia.preprocess import Preprocess

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from itertools import product
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


# Encoding

__Note__: 600 is a high value for our dataset, but is the value more mentioned in literature

# read the data is the first step !! do not forget

In [0]:
read_seqs = ReadSequence()
x_train_esm = read_seqs.par_preprocessing(dataset= x_train_esm, col = 'sequence', B ='N', Z = 'Q', U = 'C', O = 'K', J = 'I', X = '')
x_test_esm = read_seqs.par_preprocessing(dataset= x_test_esm, col = 'sequence', B ='N', Z = 'Q', U = 'C', O = 'K', J = 'I', X = '')

In [0]:
def pad_and_truncate_sequences(df,seq_col,max_length) -> pd.DataFrame:
    """
    It pads or truncates the sequences to the maximum sequence length.

    Parameters
    ----------
    x: pd.DataFrame
        The input data.

    Returns
    -------
    x: pd.DataFrame
        The padded or truncated input data.
    """

    def pad_and_truncate(seq):
        # Truncate the sequence if it's longer than max_seq_len
        if len(seq) > max_length:
            return seq[:max_length]
        # Pad the sequence if it's shorter than max_seq_len
        else:
            padding = "<pad>" * (max_length - len(seq))
            return seq + padding
    df['padded_and_truncated_sequence'] = df[seq_col].apply(pad_and_truncate)
    return df

x_train_esm_encode = pad_and_truncate_sequences(x_train_esm, 'sequence', 600)
x_test_esm_encode = pad_and_truncate_sequences(x_test_esm, 'sequence', 600)

In [0]:
enconde_train_df = Encoding(dataset= x_train_esm_encode ,  col= 'padded_and_truncated_sequence')
enconde_test_df=Encoding(dataset= x_test_esm_encode ,  col= 'padded_and_truncated_sequence')

In [0]:
esm_train = enconde_train_df.get_esm(v_esm='esm2_150')
x_train_esm=np.array([x.astype(np.float64) for x in esm_train['esm']])

esm_test = enconde_test_df.get_esm(v_esm='esm2_150')
x_test_esm=np.array([x.astype(np.float64) for x in esm_test['esm']])

In [0]:
y_train_esm=y_train_esm.drop(3274)

# Deep Learning

The aim is to recreate the deepLearning models implemented in omnia as well as the possibility of optimising hyperparameters by grid search (using a param_grid).

# RNN

In [0]:
#RNN and propythia
def create_rnn_model(rnn_type='LSTM', bidirectional=False, num_rnn_layers=1, hidden_dim=64, num_dense_layers=1, neurons_dense=32, output_dim=1, drop=0.3, activation='relu', last_layers_activations='sigmoid'):
    model = tf.keras.models.Sequential()

    # RNN layers
    for i in range(num_rnn_layers):
        current_hidden_dim =hidden_dim // (2**i)
        if rnn_type == 'LSTM':
            rnn_layer = tf.keras.layers.LSTM(current_hidden_dim, return_sequences=(i != num_rnn_layers - 1), activation=activation)
        elif rnn_type == 'GRU':
            rnn_layer = tf.keras.layers.GRU(current_hidden_dim, return_sequences=(i != num_rnn_layers - 1), activation=activation)
        elif rnn_type == 'SimpleRNN':
            rnn_layer = tf.keras.layers.SimpleRNN(current_hidden_dim, return_sequences=(i != num_rnn_layers - 1), activation=activation)
        else:
            raise ValueError("Invalid RNN type. Supported types are 'LSTM', 'GRU', and 'SimpleRNN'.")

        if bidirectional:
            rnn_layer = tf.keras.layers.Bidirectional(rnn_layer)

        model.add(rnn_layer)
        model.add(tf.keras.layers.Dropout(drop))

    # Dense layers
    for i in range(num_dense_layers):
        current_neurons_dense = neurons_dense if i == 0 else neurons_dense // (2**i)
        model.add(tf.keras.layers.Dense(current_neurons_dense, activation=activation))
        model.add(tf.keras.layers.Dropout(drop))

    model.add(tf.keras.layers.Dense(output_dim, activation=last_layers_activations))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

dl=DeepML(x_train_esm, y_train_esm, x_test_esm, y_test_esm, number_classes=2, problem_type='binary',
          x_dval=None, y_dval=None, epochs=80, batch_size=32,
          path='', report_name=None, verbose=1,
         early_stopping_patience=15, reduce_lr_patience=10, reduce_lr_factor=0.2, reduce_lr_min=0.00001,
                 )
model = KerasClassifier(build_fn=create_rnn_model)


In [0]:
%%capture captured

def generate_param_grid(num_rnn_layers:list, num_dense_layers:list):
    param_grid = {
        'rnn_type': ['LSTM', 'GRU', 'SimpleRNN'],
        'bidirectional': [True, False],
        'num_rnn_layers': num_rnn_layers,
        'hidden_dim': [64,128],
        'num_dense_layers': num_dense_layers,
        'neurons_dense': [64,128],
        'output_dim': [1],
        'drop': [0.1, 0.3],
        'activation': ['relu'],
        'last_layers_activations': ['sigmoid']
    }
    return param_grid

# Define os parâmetros do grid
num_rnn_layers = [2,3]
num_dense_layers = [2,3]
param_grid = generate_param_grid(num_rnn_layers, num_dense_layers)

# Otimização do modelo
best_classifier_rnn = dl.get_opt_params(param_grid, model, scoring=make_scorer(f1_score), optType='randomizedSearch', cv=5, n_iter_search=50)
dl.save_model(path='best_rnn_model_esm.h5', model=best_classifier_rnn)

In [0]:
scores, report, cm, cm2 = dl.score_testset_classification(best_classifier_rnn)

# Escreva cada métrica e valor em uma linha separada
with open('scores_data_esm_rnn.txt', 'w') as f:
    for metric, value in scores.items():
        f.write(f"{metric}: {value}\n")

In [0]:
y_pred = best_classifier_rnn.predict(dl.x_test)

conf_mat = confusion_matrix(dl.y_test, y_pred)
np.savetxt('confusion_matrix__data_esm2_rnn.csv', conf_mat, delimiter=",")